In [17]:
IRT_DIMS = 0
DATASET = 'matrix'    
ITEM_FEATURES = 'False'
ITEM_FEATURES = False if ITEM_FEATURES == 'False' else True

In [18]:
import sys, os, shutil
import pickle
import time
import numpy as np
from utils import (mymode, load_data_heterogeneous, create_data_object_heterogeneous)
import seaborn as sns

from IRT import MIRT_2PL
from Heterogeneous_embedder import EmbedderHeterogeneous, train_embedder_heterogeneous, test_embedder_heterogeneous
from manage_experiments import perform_cross_validation

import torch
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import pearsonr

In [19]:
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [20]:
# Initialise
parameters = {
    'df_name': None,
    'epochs': 10000,
    'learning_rate': 0.005,
    'weight_decay': 0,
    'early_stopping': 200,
    'n_splits': 10,
    'device': 'cuda:0',
    'batch_size': 2**14,
    'neighbours': [50, 50]
    }

if IRT_DIMS > 0:
    parameters['model_type'] = 'IRT'
    parameters['hidden_dims'] = IRT_DIMS
    parameters['lambda1'] = 0
    parameters['lambda2'] = 0
    OUTNAME = 'IRT'
else:
    parameters['model_type'] = 'GNN'
    parameters['hidden_dims'] = [16,8]
    OUTNAME = 'SAGE' 

if ITEM_FEATURES:
    OUTNAME = OUTNAME + '_scales'
    
print(parameters)
print(DATASET)

{'df_name': None, 'epochs': 10000, 'learning_rate': 0.005, 'weight_decay': 0, 'early_stopping': 200, 'n_splits': 10, 'device': 'cuda:0', 'batch_size': 16384, 'neighbours': [50, 50], 'model_type': 'GNN', 'hidden_dims': [16, 8]}
matrix


In [21]:
DATA_FILE = 'data/mindsteps_set_' + DATASET
df = load_data_heterogeneous(DATA_FILE)
data, df_student, df_item, df_edge = create_data_object_heterogeneous(df, return_aux_data=True, item_features=ITEM_FEATURES)

In [22]:
print(data)

HeteroData(
  student={
    node_id=[51534],
    x=[51534, 2],
  },
  item={ node_id=[17885] },
  (student, responds, item)={
    edge_index=[2, 5614949],
    edge_attr=[5614949, 2],
    y=[5614949],
  },
  (item, rev_responds, student)={ edge_index=[2, 5614949] }
)


In [6]:
df.columns

Index(['studentId', 'code', 'motherTongue', 'Gender', 'scale', 'matrix',
       'IRT_difficulty', 'topic', 'responseformat', 'textlength', 'score',
       'age', 'grade', 'viewingTime', 'timestamp', 'useCase', 'assessmentId',
       'school_location', 'school_postcode', 'canton', 'school_code',
       'ability', 'frequency', 'previous_sessions', 'years_from_start',
       'matdiff', 'matcode', 'domain'],
      dtype='object')

In [7]:
df.head()

,studentId,code,motherTongue,Gender,scale,matrix,IRT_difficulty,topic,responseformat,textlength,...,school_postcode,canton,school_code,ability,frequency,previous_sessions,years_from_start,matdiff,matcode,domain
0,0,0,1.0,0.0,fhoe,F.1.A.1.c.1,-0.370330,Monologe und Dialoge,NaN,NaN,...,4057,Basel-Stadt,04-052,1.083005,1.851849,2,1.08,,,f
1,0,1,1.0,0.0,fles,F.2.A.1.b.3,-0.494880,Berichte,NaN,NaN,...,4057,Basel-Stadt,04-052,0.657010,2.727266,3,1.10,,,f
2,0,2,1.0,0.0,mzuv,MA.1.A.3.f,0.368830,Gemischte Grundoperationen,single choice,NaN,...,4057,Basel-Stadt,04-052,0.751265,1000000.000000,0,0.00,,,m
3,0,3,1.0,0.0,fhoe,F.1.A.1.d.1,0.367442,Aus TV und Radio,NaN,NaN,...,4057,Basel-Stadt,04-052,1.083005,1.851849,2,1.08,,,f
4,0,4,1.0,0.0,fles,F.2.A.1.d.1,1.160150,Berichte,NaN,212.0,...,4057,Basel-Stadt,04-052,0.657010,2.727266,3,1.10,,,f


edge_dim = data['student', 'responds', 'item'].edge_attr.shape[1]
if IRT_DIMS > 0:
    model = MIRT_2PL(IRT_DIMS, edge_dim, data)
else:
    model = EmbedderHeterogeneous( 
        n_students =  data['student'].x.size(0),
        n_items = data['item'].x.size(0),
        student_inchannel = data['student'].x.size(1),
        item_inchannel = data['item'].x.size(1),
        hidden_channels=parameters['hidden_dims'],
        edge_channel=edge_dim,
        metadata=data.metadata()
        ).to(device)
    
model = model.to(device)

In [ ]:
output_dict, model = perform_cross_validation(data, parameters, save_embeddings=True, save_subgraph=True, final_fit=True)

0it [00:00, ?it/s]
  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:01<00:03,  1.15s/it]

 50%|█████     | 2/4 [00:02<00:01,  1.00it/s]

 75%|███████▌  | 3/4 [00:02<00:00,  1.11it/s]

100%|██████████| 4/4 [00:03<00:00,  1.13it/s]



Epoch: 001, Loss: 0.6789, Val: 0.4999, Test: 0.4984



  0%|          | 1/10000 [00:05<14:59:48,  5.40s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:01<00:03,  1.14s/it]

 50%|█████     | 2/4 [00:02<00:01,  1.01it/s]

 75%|███████▌  | 3/4 [00:02<00:00,  1.09it/s]

100%|██████████| 4/4 [00:03<00:00,  1.10it/s]



Epoch: 002, Loss: 0.6690, Val: 0.5029, Test: 0.5012



  0%|          | 2/10000 [00:10<15:08:37,  5.45s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:01<00:03,  1.20s/it]

 50%|█████     | 2/4 [00:02<00:02,  1.01s/it]

 75%|███████▌  | 3/4 [00:02<00:00,  1.06it/s]

100%|██████████| 4/4 [00:03<00:00,  1.07it/s]


...


  3%|▎         | 268/10000 [19:18<11:00:57,  4.07s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:02,  1.19it/s]

 50%|█████     | 2/4 [00:01<00:01,  1.20it/s]

 75%|███████▌  | 3/4 [00:02<00:00,  1.22it/s]

100%|██████████| 4/4 [00:03<00:00,  1.26it/s]

  3%|▎         | 269/10000 [19:22<10:49:03,  4.00s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:02,  1.25it/s]

 50%|█████     | 2/4 [00:01<00:01,  1.27it/s]

 75%|███████▌  | 3/4 [00:02<00:00,  1.26it/s]

100%|██████████| 4/4 [00:03<00:00,  1.30it/s]

  3%|▎         | 270/10000 [19:26<10:32:52,  3.90s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:02,  1.33it/s]

 50%|█████     | 2/4 [00:01<00:01,  1.28it/s]

 75%|███████▌  | 3/4 [00:02<00:00,  1.28it/s]

100%|██████████| 4/4 [00:03<00:00,  1.32it/s]

  3%|▎         | 271/10000 [19:29<10:20:53,  3.83s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:02,  1.28it/s]

 50%|███


Epoch: 279, Loss: 0.3798, Val: 0.7140, Test: 0.7138



  3%|▎         | 279/10000 [20:00<10:51:19,  4.02s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:02,  1.18it/s]

 50%|█████     | 2/4 [00:01<00:01,  1.21it/s]

 75%|███████▌  | 3/4 [00:02<00:00,  1.21it/s]

100%|██████████| 4/4 [00:03<00:00,  1.26it/s]

  3%|▎         | 280/10000 [20:03<10:38:26,  3.94s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:02,  1.23it/s]

 50%|█████     | 2/4 [00:01<00:01,  1.21it/s]

 75%|███████▌  | 3/4 [00:02<00:00,  1.21it/s]

100%|██████████| 4/4 [00:03<00:00,  1.25it/s]



Epoch: 281, Loss: 0.3791, Val: 0.7141, Test: 0.7138



  3%|▎         | 281/10000 [20:08<11:14:40,  4.17s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:02,  1.14it/s]

 50%|█████     | 2/4 [00:01<00:01,  1.16it/s]

 75%|███████▌  | 3/4 [00:02<00:00,  1.21it/s]

100%|██████████| 4/4 [00:03<00:00,  1.24it/s]

  3%|▎         | 282/10000 [20:12<11:02:13,  4.09s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:02,  1.25it/s]

 50%|█████     | 2/4 [00:01<00:01,  1.27it/s]

 75%|███████▌  | 3/4 [00:02<00:00,  1.27it/s]

100%|██████████| 4/4 [00:03<00:00,  1.29it/s]

  3%|▎         | 283/10000 [20:16<10:41:59,  3.96s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:02,  1.30it/s]

 50%|█████     | 2/4 [00:01<00:01,  1.23it/s]

 75%|███████▌  | 3/4 [00:02<00:00,  1.20it/s]

100%|██████████| 4/4 [00:03<00:00,  1.24it/s]

  3%|▎         | 284/10000 [20:19<10:36:11,  3.93s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:02,  1.37it/s]

 50%|███

In [15]:
output_dict.keys()
print('AUC:', output_dict['AUC_0_test'])
print('Balanced Accuracy:', output_dict['Balanced Accuracy_0_test'])

AUC: 0.7899070881929522
Balanced Accuracy: 0.7139247676778068


In [16]:
unique_scales = df_item['scale'].unique()
unique_domains = df_item['domain'].unique()
unique_matdiff = df_item['matdiff'].sort_values().unique()
#scale_colors = dict([(c, plt.cm.tab10(i)) for i, c in enumerate(unique_scales)])
#domain_colors = dict([(c, plt.cm.tab10(i)) for i, c in enumerate(unique_domains)])

In [ ]:
train_losses = output_dict['losses_0']
train_edge_indices, val_edge_indices, test_edge_indices = output_dict['indices_0']

train_data = output_dict['train_subgraph_data'] 
val_data = output_dict['val_subgraph_data'] 
test_data = output_dict['test_subgraph_data']

aux_data = (df, df_student, df_item, df_edge, 
    #clustering_indices, 
    train_losses, #test_losses, test_aucs, 
    train_edge_indices, val_edge_indices, test_edge_indices, 
    data.cpu(), train_data.cpu(), val_data.cpu(), test_data.cpu(),
    unique_scales, unique_domains, unique_matdiff, 
    DATA_FILE)

with open(f'./results/{OUTNAME}_{DATASET}_aux_data.pkl', 'wb') as handle:
    pickle.dump(aux_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# SAVE EVERYTHING
torch.save(model, f'./results/{OUTNAME}_{DATASET}.pth.tar')

In [ ]:
train_indices = np.arange(len(train_losses))
fig = plt.figure()
plt.plot(train_indices, train_losses, c='blue')
plt.ylabel('Loss')
plt.xlabel('Epoch')

fig.tight_layout()

In [ ]:
# save IRT parameters to matrix
if OUTNAME == 'IRT' and IRT_DIMS == 1:    
    #z_dict = model.get_embeddings(train_data.to(device))
    z_dict = output_dict['embedding_0']
    df_item['IRT1_difficulty'] = -z_dict['offset']
    df_item['IRT1_discrimination'] = z_dict['discrimination']
    df_item['IRT1_discrimination_transf'] = z_dict['item']
    ability = z_dict['ability']
    df_edge['IRT1_ability'] = ability.ravel()
    
    aux_data = (df, df_student, df_item, df_edge, 
    train_losses, 
    train_edge_indices, val_edge_indices, test_edge_indices,
    data.cpu(), train_data.cpu(), val_data.cpu().cpu(), test_data.cpu(),
    unique_scales, unique_domains, unique_matdiff, 
    DATA_FILE)

    with open(f'./results/{OUTNAME}_{DATASET}_aux_data_IRT1.pkl', 'wb') as handle:
        pickle.dump(aux_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    df_item_clean = df_item.dropna(subset=['IRT_difficulty', 'IRT1_difficulty'])    
    
    fig = plt.figure()
    sns.scatterplot(x='IRT_difficulty', y='IRT1_difficulty', data=df_item, hue='scale')
    plt.title('Difficulty')
    print('Difficulty:', pearsonr(df_item_clean['IRT1_difficulty'], df_item_clean['IRT_difficulty']))
    
    edge_feat = train_data['student', 'responds', 'item'].edge_attr.detach().cpu().numpy()
    df_edge_clean = df_edge.dropna(subset=['IRT1_ability', 'ability', 'age'])
    fig = plt.figure()
    sns.scatterplot(x='age', y='IRT1_ability', data=df_edge_clean, hue='grade')
    plt.title('Age-Ability')
    
    fig = plt.figure()
    sns.scatterplot(x='grade', y='IRT1_ability', data=df_edge_clean, hue='age')
    plt.title('Grade-Ability')
    print('Age-Ability:', pearsonr(df_edge_clean['age'], df_edge_clean['IRT1_ability']))

    fig = plt.figure()
    sns.scatterplot(x='ability', y='IRT1_ability', data=df_edge_clean, hue='grade')
    plt.title('Ability')
    print('Ability:', pearsonr(df_edge_clean['ability'], df_edge_clean['IRT1_ability']))
    